# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,15.09,70,4,8.87,CL,1711741616
1,1,dali,25.7000,100.1833,9.85,90,24,2.27,CN,1711741733
2,2,puerto natales,-51.7236,-72.4875,5.25,81,75,4.63,CL,1711741734
3,3,dapa,9.7594,126.0531,25.94,74,3,2.83,PH,1711741734
4,4,iqaluit,63.7506,-68.5145,-10.15,85,100,2.57,CA,1711741735


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# city_data_df['Humidity'] = city_data_df['Humidity'].astype(int).abs()

In [17]:
# Configure the map plot



map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    size = 'Humidity', 
    color = 'City',
    tiles = 'OSM',
    frame_width = 750,
    frame_height = 750
    
)


# Display the map

map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_high_temp = city_data_df[city_data_df['Max Temp']<35] 
ideal_temp = ideal_high_temp[ideal_high_temp['Max Temp']>25]
ideal_humidity = ideal_temp[ideal_temp['Humidity']<70]
ideal_cloudiness = ideal_humidity[ideal_humidity['Cloudiness']<50]
ideal_weather = ideal_cloudiness[ideal_cloudiness['Wind Speed']<4]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
145,145,cabo san lucas,22.8909,-109.9124,29.23,50,0,3.25,MX,1711741813
196,196,mandasa,18.8667,84.4667,26.36,65,1,3.82,IN,1711741843
202,202,baud,20.8333,84.3167,29.55,40,3,1.06,IN,1711741846
206,206,brooksville,28.5553,-82.3879,26.55,32,0,3.09,US,1711741672
220,220,alaghsas,17.0187,8.0168,31.87,5,10,3.97,NE,1711741857
243,243,boende,-0.2167,20.8667,26.48,62,33,1.05,CD,1711741869
247,247,illizi,26.4833,8.4667,27.89,6,0,2.56,DZ,1711741871
263,263,erandol,20.9167,75.3333,29.35,38,6,2.27,IN,1711741881
269,269,tomatlan,19.9333,-105.2500,32.33,30,47,3.99,MX,1711741885
296,296,san ignacio,17.1561,-89.0714,30.08,51,0,3.09,BZ,1711741897


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
145,cabo san lucas,MX,22.8909,-109.9124,50,
196,mandasa,IN,18.8667,84.4667,65,
202,baud,IN,20.8333,84.3167,40,
206,brooksville,US,28.5553,-82.3879,32,
220,alaghsas,NE,17.0187,8.0168,5,
243,boende,CD,-0.2167,20.8667,62,
247,illizi,DZ,26.4833,8.4667,6,
263,erandol,IN,20.9167,75.3333,38,
269,tomatlan,MX,19.9333,-105.2500,30,
296,san ignacio,BZ,17.1561,-89.0714,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
   
    # get latitude, longitude from the DataFrame    
    lng = hotel_df.loc[index,'Lng']
    lat = hotel_df.loc[index,'Lat']

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
    'categories': 'accommodation.hotel',
    'filter': f'circle:{lng},{lat},{radius}',
    'bias': f'proximity:{lng},{lat}',
    'limit': 20,
    'apiKey':geoapify_key
}

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
mandasa - nearest hotel: No hotel found
baud - nearest hotel: No hotel found
brooksville - nearest hotel: No hotel found
alaghsas - nearest hotel: Hôtel Zagado
boende - nearest hotel: Hotel Reference
illizi - nearest hotel: بوناقة
erandol - nearest hotel: No hotel found
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
san ignacio - nearest hotel: Rainforest Haven Inn
sao vicente - nearest hotel: Pousada Vitória
santo tome - nearest hotel: Colon Hotel Resort & Spa
eusebio ayala - nearest hotel: San Francisco Country Club
coxim - nearest hotel: Avendita
guaruja - nearest hotel: Ferrareto Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
145,cabo san lucas,MX,22.8909,-109.9124,50,Comfort Rooms
196,mandasa,IN,18.8667,84.4667,65,No hotel found
202,baud,IN,20.8333,84.3167,40,No hotel found
206,brooksville,US,28.5553,-82.3879,32,No hotel found
220,alaghsas,NE,17.0187,8.0168,5,Hôtel Zagado
243,boende,CD,-0.2167,20.8667,62,Hotel Reference
247,illizi,DZ,26.4833,8.4667,6,بوناقة
263,erandol,IN,20.9167,75.3333,38,No hotel found
269,tomatlan,MX,19.9333,-105.2500,30,Hotel Hacienda Vieja Tomatlan
296,san ignacio,BZ,17.1561,-89.0714,51,Rainforest Haven Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE


map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size = 'Humidity',
    color = 'City',
    frame_width = 750,
    frame_height = 750,
    hover_cols=['Country', 'Hotel Name'],
)


# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)